# Assignment Instructions
  
1. Before you submit this assignment for grading, make sure everything runs as expected. 
    1. **Restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart)
    2. Then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).  
2. Fill in your STUDENTID (abc123) in the code block below.
3. Make sure you fill in any place that says `#YOUR CODE HERE` or "YOUR ANSWER HERE"
4. When filling in `#YOUR CODE HERE` sections, remove or comment out the line  
  - `raise NotImplementedError()`

By submitting this notebook for grading, you affirm that all work was produced by the author identified below, and that references are included for all use of public source material (to include code, data, diagrams, pictures, and verbatim text).

In [1]:
STUDENTID = "tfs436"

---

# AS01 - Review of Python Libraries
**Version:**  1.0  
**Total Points:**  5.0

## Objective
The objective of this assignment is to review some useful basics of Python, Pandas, and Matplotlib.

## Background
You may find the following references helpful

| Documentation | URL |
| :--- | :--- |
| Python | https://docs.python.org/3.7/library/index.html |
| Pandas | https://pandas.pydata.org/pandas-docs/stable/reference/index.html |
| Matplotlib | https://matplotlib.org/api/_as_gen/matplotlib.pyplot.html#module-matplotlib.pyplot |
| Numpy | https://docs.scipy.org/doc/numpy/reference/index.html |
| Seaborn | http://seaborn.pydata.org/ |

## Setup
Put imports up front and use consistent nomenclature for `as` names

In [2]:
# Imports and globals for this exercise
# Setup notebook for interactive graphics
%matplotlib notebook
# mainline tools
import os
import math
import re
import numpy as np
import pandas as pd
import scipy.stats as st
# graphics
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib as mpl
import seaborn
# Cartograpy support
import mplleaflet as mlf
# indirectly referenced libraries
import pyarrow
import fastparquet

# Ensure the Dataset root is available
sDataRoot = os.environ['DATASETS_ROOT']
assert os.path.exists(os.path.join(sDataRoot,'readme.txt'))

# Uniform place to save temporary data and avoid lengthy reprocessing
sTempDir = os.path.join(os.environ['HOMEDRIVE'], os.environ['HOMEPATH'],'_tmp')
if not os.path.exists(sTempDir):
    os.makedirs(sTempDir)


## Python Review


### Pythagorean Theorum (1 point)
Write a function that calculates the hypotenuse of a triangle given the length of 2 sides.  
The formual is $c = \sqrt{a^2 + b^2}$  
Make sure it handles the case where $a<0$ or $b<0$ by raising a `ValueError`.

In [3]:
def pythagoras(a, b):
    """Compute the hypotenuse of a triangle given the lenght of two sides (a,b)
    """
    return math.sqrt(a**2 + b**2)

In [4]:
## This is an automatically graded test cell.
# It contains public tests that you can use to help determine whether your
# functions are correct. It also contains hidden tests that are run by
# the autograder.

# Public tests
# ---------------------------------------------------------
np.testing.assert_almost_equal(pythagoras(1,1), 1.4142, decimal=4, err_msg='Almost equality failed', verbose=True)
np.testing.assert_almost_equal(pythagoras(3,4), 5.0, decimal=4, err_msg='Almost equality failed', verbose=True)
assert '{:.4f}'.format(pythagoras(2,3)) == '3.6056'


## Pandas Review
For these problems, we'll use a dataset obtained from the [Worldbank education statistics](https://databank.worldbank.org/source/education-statistics-^-all-indicators/)  
The dataset contains education information for 264 countries for the years 1990 to 2018.

- The files are located in the DataSets directory.
- You might want to open the files in Excel to get a feel for the organization and headers.



In [5]:
# These are the files names
sMetaFile = os.path.join(sDataRoot,'Worldbank','Education','09c8eeb5-e243-4ed7-addd-818659c2f5f3_Series - Metadata.csv')
sDataFile = os.path.join(sDataRoot,'Worldbank','Education','09c8eeb5-e243-4ed7-addd-818659c2f5f3_Data.csv')
assert os.path.exists(sMetaFile)                         
assert os.path.exists(sDataFile)


### Read the metadata file
The first function must return a pandas series. 
- only read in the first two columns
- The series must have the index name `Code` and the seris name `IndicatorName` 

The second function is for looking up the correct code
- It must return a list
- Find all the entries in the `IndicatorName` that match all the words in the input string, case insensitive
- You might want to use the regular expression version of split to eliminate punctuation

In [9]:
from functools import reduce
def wdbe_read_metadata(sMetaFile):
    '''
    Read in a CSV metadata data file exported from the World Data Bank Education dataset.
    https://databank.worldbank.org/source/education-statistics-%5E-all-indicators/
    '''
    return pd.read_csv(sMetaFile,index_col = 0,usecols = ['Code','Indicator Name'])
    
def wdbe_lookup_code( srm, sIndicator ):
    '''
    Return a list of codes where the 'Indicator Name' contains all the keywords supplied in the sIndicator string
    '''
    sIndicator_split = re.split('[,()\s]',sIndicator)
    sIndicator_split = list(filter(None,sIndicator_split))
    df = srm[reduce(lambda a, b: a&b, (srm['Indicator Name'].str.contains(indicators,case = False) for indicators in sIndicator_split))]
    return df.index

                                                      Indicator Name
Code                                                                
SE.XPD.TOTL.GD.ZS  Government expenditure on education as % of GD...
SE.PRM.ENRR.FE            Gross enrolment ratio, primary, female (%)
SE.ENR.PRIM.FM.ZS  Gross enrolment ratio, primary, gender parity ...
SE.PRE.NENR.FE           Net enrolment rate, pre-primary, female (%)
SE.PRM.NENR.FE               Net enrolment rate, primary, female (%)
SE.PRM.UNER        Out-of-school children of primary school age, ...
SE.PRM.CMPT.ZS               Primary completion rate, both sexes (%)
SE.ADT.LITR.ZS     Adult literacy rate, population 15+ years, bot...
SE.ADT.LITR.FE.ZS  Adult literacy rate, population 15+ years, fem...
SE.ADT.LITR.MA.ZS  Adult literacy rate, population 15+ years, mal...
SE.PRM.TENR.MA        Adjusted net enrolment rate, primary, male (%)
UIS.NERA.1.GPI     Adjusted net enrolment rate, primary, gender p...
SE.PRM.TENR.FE      Adjusted net e

In [7]:
## This is an automatically graded test cell.
# It contains public tests that you can use to help determine whether your
# functions are correct. It also contains hidden tests that are run by
# the autograder.

# Use the defined functions
srm = wdbe_read_metadata(sMetaFile)

# Public tests
# ---------------------------------------------------------
assert 6 == len(wdbe_lookup_code(srm, 'net enrolment'))
assert sorted(['SE.ADT.LITR.FE.ZS']) == sorted(wdbe_lookup_code(srm, 'adult literacy female'))


                                                      Indicator Name
Code                                                                
SE.XPD.TOTL.GD.ZS  Government expenditure on education as % of GD...
SE.PRM.ENRR.FE            Gross enrolment ratio, primary, female (%)
SE.ENR.PRIM.FM.ZS  Gross enrolment ratio, primary, gender parity ...
SE.PRE.NENR.FE           Net enrolment rate, pre-primary, female (%)
SE.PRM.NENR.FE               Net enrolment rate, primary, female (%)
SE.PRM.UNER        Out-of-school children of primary school age, ...
SE.PRM.CMPT.ZS               Primary completion rate, both sexes (%)
SE.ADT.LITR.ZS     Adult literacy rate, population 15+ years, bot...
SE.ADT.LITR.FE.ZS  Adult literacy rate, population 15+ years, fem...
SE.ADT.LITR.MA.ZS  Adult literacy rate, population 15+ years, mal...
SE.PRM.TENR.MA        Adjusted net enrolment rate, primary, male (%)
UIS.NERA.1.GPI     Adjusted net enrolment rate, primary, gender p...
SE.PRM.TENR.FE      Adjusted net e

TypeError: 'NoneType' object is not subscriptable

### Read the data file and organize data (1 point)
Complete the following function to read in the data file as a pandas dataframe. You might want to brush up on the pd.read_csv function parameters as you will need to use several of them.  
- The column names for the years should be adjusted to just be the year as an integer
- Note that NaN values should be included, and they are encoded as ..
- Do not include the "Series" column
- There are some comments at the end of the file that you will have to exclude
- Note that the data file is encoded as UTF-8
- The row order in the dataframe must be the same as the csv file

The result will be a dataframe with shape (3696, 32)

In [ ]:
def wdbe_read_data(sFile):
    '''
    Read in a CSV data file exported from the World Data Bank Education dataset.
    https://databank.worldbank.org/source/education-statistics-%5E-all-indicators/
    '''
    #url.split('.',1)[0]
    df = pd.read_csv(sFile,na_values = '..',encoding = 'UTF-8',nrows = 3696)
    df = df.drop(['Series'], axis=1)
    new_cols = np.arange(1990,2019,1)
    df.rename(columns=dict(zip(df.columns[3:], new_cols)),inplace=True)
    return (df)
#wdbe_read_data(sDataFile)

In [ ]:
## This is an automatically graded test cell.
# It contains public tests that you can use to help determine whether your
# functions are correct. It also contains hidden tests that are run by
# the autograder.

# Use the defined functions
dfData = wdbe_read_data(sDataFile)

# Public tests
# ---------------------------------------------------------
assert dfData.shape == (3696, 32)
assert np.isnan(dfData[ (dfData['Country Code']=='LKA') & (dfData['Series Code'] == 'SE.PRM.TENR.MA') ].reset_index(drop=True).at[0,1990])
                          

## Explore the data
Given a 'Series Code' and a number of years (n), return a sorted list of country codes that have at least n years of data (not nan) available.
- The returned value must be a list of strings
- The list must be sorted alphabetically (A-Z)

In [ ]:
def wdbe_available_data(dfData, sSeries, nYrs):
    '''
    Find the country codes for rows that have at least nYrs of data for
    the given series code.
    '''
    df = dfData[dfData['Series Code'] == sSeries]
    df.loc[:,'Count'] = df.iloc[:,3:].count(axis=1)
    df = df[df['Count']>=nYrs]
    return (sorted(df['Country Code']))
#wdbe_available_data(dfData, 'SE.XPD.TOTL.GD.ZS', 25)

In [ ]:
## This is an automatically graded test cell.
# It contains public tests that you can use to help determine whether your
# functions are correct. It also contains hidden tests that are run by
# the autograder.

# Public tests
# ---------------------------------------------------------
assert isinstance(wdbe_available_data(dfData, 'SE.XPD.TOTL.GD.ZS', 25), list)
assert ['CHE', 'CIV', 'ESP', 'IRL', 'IRN', 'JAM', 'NOR'] == wdbe_available_data(dfData, 'SE.XPD.TOTL.GD.ZS', 25)


## Extract data
Given a series code, a list of country codes, and a minimum number of data points required, return a dataframe with the year as the row index value, and where each country that meets the minimum is a column with the country name (not country code) as the column name.
- The result must be a dataframe
- The name of the index must be "Year"
- Return all years in the table. Missing data should be NaN.
- The name of the dataframe columns must be "Country Name" 

In [ ]:
def wdbe_extract_data(dfData, lsCountries, sSeries, nYrs ):
    '''
    Extract data from the education dataframe given:  
    dfData - the education dataframe  
    lsCountries - list of country codes
    sSeries - single series name
    nYrs - must have at least this many years of data
    '''
    df = dfData[(dfData['Country Code'].isin(lsCountries)) & (dfData['Series Code'] == sSeries)]
    df.loc[:,'Count'] = df.iloc[:,3:].count(axis=1)
    df = df[df['Count']>=nYrs]
    df = df.drop(['Series Code','Country Code','Count'], axis=1)
    df = df.set_index('Country Name')
    df1_transposed = df.T
    df1_transposed.index.name='Year'
    df1_transposed.columns.name="Country Name"
    return (df1_transposed)

In [ ]:
## This is an automatically graded test cell.
# It contains public tests that you can use to help determine whether your
# functions are correct. It also contains hidden tests that are run by
# the autograder.

# Call the functions
sIndicator = 'Gross enrolment ratio, primary, gender parity index (GPI)'
sCode = wdbe_lookup_code( srm, sIndicator )[0]
assert 'SE.ENR.PRIM.FM.ZS' == sCode
lsCountries = ['USA', 'CHN', 'RUS' ]
nYrs = 20
dfData = wdbe_read_data(sDataFile)
dfGPI = wdbe_extract_data(dfData, lsCountries, sCode, nYrs )

# Public tests
# ---------------------------------------------------------
assert 'Country Name' == dfGPI.columns.name
assert 'Year' == dfGPI.index.name
assert (29,3) == dfGPI.shape
np.testing.assert_almost_equal(dfGPI['China'].mean(), 0.973437, decimal=6, err_msg='Almost equality failed', verbose=True)


## Plot Data
Use the results from the previous extract (dfGPI) to plot the literacy rates as a function of year. The next cell produces a quick plot using the dataframe.plot call. Notes of interest:
- it automatically includes a legend based on the column names
- it labels the x axis using the index name
- it does not label the y axis or automatically title the graph
- missing data is skipped

You might want to check out how this looks if you do not set the y axis limits.

In [ ]:
# Use the pandas plot to produce a quick, rough plot
ax = dfGPI.plot.line(title=sIndicator, figsize=[7,5])
ax.set_ylim([0.5,1.1])
ax.set_ylabel('GPI')

Now produce essentially the same plot as above but use matplotlib with the following modifications.
- put minor ticks on the x axis for each year
- offset 1990 from the y axis so the first data point is not right on the axis

This will be manually graded according to the following rubric:
- Plot has correct title, legend, and labels on both axes (0.25 points)
- Move the legend to the lower right corner (0.25)
- 1990 is offset a little from the y axis (0.25 points)
- x axis has minor ticks at 1 year intervals (0.25 points)


In [ ]:
h = plt.figure(figsize=[7,5])
ax = plt.gca()
ax.plot(dfGPI)
plt.title('Gross enrolment ratio, primary, gender parity index (GPI)')
plt.xlabel('Year')
plt.ylabel('GPI')
plt.legend(dfGPI.columns.values,title = 'Country Name',loc='lower right')
ax.set_ylim([0.5,1.1])
ax.set_xlim([1989,2018])
ax.set_xticks(np.arange(1990,2019), minor=True)
plt.show()
